In [15]:
import os
from dotenv import load_dotenv

from langchain_gigachat.chat_models import GigaChat
from langchain_huggingface import HuggingFaceEndpoint
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
import bs4
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import PromptTemplate

load_dotenv()

True

In [2]:
giga_key = os.environ.get("GIGACHAT_CREDENTIALS")

## Model

In [3]:
giga = GigaChat(credentials=giga_key, model="GigaChat", timeout=30, verify_ssl_certs=False)
giga.verbose = False

In [8]:
giga.invoke('Hello')

AIMessage(content='Привет! Как могу помочь?', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 12, 'completion_tokens': 8, 'total_tokens': 20}, 'model_name': 'GigaChat:1.0.26.20', 'finish_reason': 'stop'}, id='run-65431621-8dd8-4d12-9453-c6c7412c2556-0')

## Document loaders

In [4]:
loader = CSVLoader('../data/wine_desc.csv', source_column='Название')
data = loader.load()

In [5]:
data[:2]

[Document(metadata={'source': 'Вино "Кремснер Хунд" ', 'row': 0}, page_content='Название: Вино "Кремснер Хунд"\nСредняя цена: 18915\nДегустационные характеристики: Вино темно-золотистого цвета с насыщенным, сложным букетом, в котором преобладают тона спелого абрикоса, персика, мандарина и жимолости. Полное, с мягкой кислотностью, во вкусе раскрываются ароматы фруктов и специй, а в длинном послевкусии звучат минеральные ноты.\nИнтересные факты: Вино "Кремснер Хунд" производится из винограда рислинг, выращенного на известковых склонах Вахау. Это один из первых белых вин Австрии, которое получило международное признание. Вина Кремснер Хунд пользуются популярностью у ценителей вина всего мира.\nСпособ выдержки: В течение 9 месяцев в дубовых бочках.\nСпособ ферментации: Ферментация проводится в течение 20 дней в нержавеющей стали.'),
 Document(metadata={'source': 'Шардоне "Лес Баттарда-Монтреш" 2018', 'row': 1}, page_content='Название: Шардоне "Лес Баттарда-Монтреш" 2018\nСредняя цена: 1950

## Embeddings

In [6]:
embeddings_model = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-small",
    multi_process=True,
    encode_kwargs={"normalize_embeddings": True})

## Vector store

In [7]:
db = Chroma.from_documents(data, embeddings_model)

In [8]:
query = "Что выбрать в качестве десертного вина?"
docs = db.similarity_search(query, k=5)

In [14]:
print(docs[0].page_content)

Название: Токайски Десертний Вино
Средняя цена: 15000
Дегустационные характеристики: Вино золотисто-янтарного цвета с насыщенным и сложным букетом, в котором преобладают ноты тропических фруктов, мёда и сухофруктов. Сладкое, с хорошей кислотностью, во вкусе раскрывается ароматами абрикосов, персиков и мёда, а в длинном послевкусии звучат ноты ванили и сухого финика.
Интересные факты: Это сладкое белое вино было создано в 1650г. и по праву считается одним из лучших десертных вин в мире. Изготавливается из винограда сортов Фурминт, Харшлевелю и Зета. Вино обладает уникальным букетом и может храниться до 50 лет
Способ выдержки: 2 года в дубовых бочках.
Способ ферментации: Ферментация проводится в течение 14-16 дней в дубовых бочках.


## RAG Chain

In [27]:
template = (
    "Ты являешься профессиональным сомелье и помощником по подбору вин. "
    "Используй приведенные ниже фрагменты из извлеченного контекста, чтобы ответить на вопрос и помочь пользователю. "
    "Если ты не знаете ответа, просто скажи, что ты не знаешь. "
    "Предлагай максимум три варианта.\n"
    "Вопрос: {question}\n"
    "Контекст: {context}\n"
    "Ответ:"
)

prompt = PromptTemplate(template=template, input_variables=['context', 'question'])

In [28]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = db.as_retriever()
# prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | giga
    | StrOutputParser()
)

In [29]:
print(rag_chain.invoke('Что выбрать в качестве десертного вина?'))

Исходя из предоставленных описаний, можно предложить следующие варианты:

1. **Токайски Десертний Вино** – это вино обладает богатым букетом с нотами тропических фруктов, мёда и сухофруктов, что делает его отличным выбором для тех, кто предпочитает сложные и насыщенные десертные вина. Кроме того, оно обладает высокой плотностью и сахаристостью, что позволяет ему долго храниться.

2. **Вино "Кремарто Десерт Вино"** – имеет менее выраженный, но все же сладкий букет с тонами спелых фруктов и специй, которые могут понравиться любителям более тонких и деликатных десертных вин. Также оно отличается хорошим балансом между сладостью и кислотностью.

3. **Грасса де Поджо** – несмотря на свою высокую цену, это вино выделяется своим сложным и многослойным букетом, сочетающим ароматы сухофруктов, орехов и специй. Оно может стать прекрасным выбором для ценителей утонченных и сложных десертных вин.


In [30]:
print(rag_chain.invoke('Посоветуй что-нибудь к морепродуктам на ужин'))

К морепродуктам отлично подойдет белое вино **Шабли Мишель Лассер** от известного винодела Мишеля Лассера. Это вино производится из винограда сорта Шардоне и обладает ярким вкусом, прекрасно сочетающимся с блюдами из морепродуктов.
